# A.1 - What is PyTorch?

## A.1.3 - Installing PyTorch

In [ ]:
# Import PyTorch and check version
import torch
torch.__version__ # 2.2.2

In [ ]:
# Check CUDA availability
torch.cuda.is_available() # False

In [ ]:
# Check MPS availability (for Apple M1 chip)
print(torch.backends.mps.is_available()) # False

## Exercise A.1 - Page 257
Finished installing and setting up PyTorch on my computer.

## Exercise A.2 - Page 257
Finished running the supplementary code and everything outputed correctly. My environment is set up correctly.

# A.2 - Understanding tensors

## A.2.1 - Scalars, vectors, matrices, and tensors

In [ ]:
# Initialize tensors with various dimensions
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2], [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])


## A.2.2 - Tensor data types

In [ ]:
# Tensor data types
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

In [ ]:
# Intialize float tensor
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

In [ ]:
# Convert to float data type
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

## A.2.3 - Common PyTorch tensor operations

In [ ]:
# Initialize a tensor
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print(tensor2d)

In [ ]:
# Tensor shape - 2 rows and 3 columns
print(tensor2d.shape)

In [ ]:
# Tensor reshaping
print(tensor2d.reshape(3, 2))

In [ ]:
# Tensor views
print(tensor2d.view(3, 2))

In [ ]:
# Tensor transposing
print(tensor2d.T)

In [ ]:
# Tensor multiplication using matmul
print(tensor2d.matmul(tensor2d.T))

In [ ]:
# Tensor multiplication using @
print(tensor2d @ tensor2d.T)

# A.3 - Seeing models as computation graphs

In [ ]:
# Seeing models as computation graphs
import torch.nn.functional as F
y = torch.tensor([1.0])             # True label
x1 = torch.tensor([1.1])            # Input feature
w1 = torch.tensor([2.2])            # Weight parameter
b = torch.tensor([0.0])             # Bias unit
z = x1 * w1 + b                     # Net input
a = torch.sigmoid(z)                # Activation and output
loss = F.binary_cross_entropy(a, y)
print(f"The loss is {loss}.")

# A.4 - Automatic differentiation made easy

In [ ]:
# Computing gradients via autograd
from torch.autograd import grad
y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

In [ ]:
# Using grad function manually
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)
grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)
print(grad_L_w1)
print(grad_L_b)

In [ ]:
# Automated process - PyTorch compute the gradients of all the leaf nodes in this graph.
loss.backward()
print(w1.grad)
print(b.grad)

# A.5 - Implementing multilayer NN

In [ ]:
# A multilayer perceptron with two hidden layers
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(num_inputs, 30), # 1st hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(30, 20), # 2nd hidden layer
            torch.nn.ReLU(),
            torch.nn.Linear(20, num_outputs) # output layer
        )
    
    def forward(self, x):
        return self.layers(x)

In [ ]:
# Print model description
model = NeuralNetwork(50, 3)
print(model)

In [ ]:
# Print the total number of trainable model parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable model parameters: {num_params}")

In [ ]:
# Print the weight matrix - Random output each time
print(model.layers[0].weight)

In [ ]:
# Print the shape of the weight matrix
print(model.layers[0].weight.shape)

In [ ]:
# Print the bias
print(model.layers[0].bias)

In [ ]:
# Print the size for bias
print(model.layers[0].bias.shape)

In [ ]:
# Make the random number initialization reproducible
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

In [ ]:
# A forward pass
torch.manual_seed(123)
X = torch.rand((1, 50))
output = model(X)
print(output)

In [ ]:
# For prediction after training
with torch.no_grad():
    output = model(X)
print(output)

In [ ]:
# Compute class-membership probabilities
with torch.no_grad():
    output = torch.softmax(model(X), dim=1)
print(output)

# A.6 - Setting up efficient data loaders

In [ ]:
# Creating a small toy dataset
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

In [ ]:
# Defining a custom Dataset class
from torch.utils.data import Dataset
class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
    
    def __getitem__(self, index):
        this_x = self.features[index]
        this_y = self.labels[index]
        return this_x, this_y
    
    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)
print(len(train_ds))

In [ ]:
# Instantiating data loaders
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0 # The number of background processes
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [ ]:
# Print the training data from data loader
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

In [ ]:
# Drop the last batch
train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0, # The number of background processes
    drop_last = True
)

In [ ]:
# Print the training data from data loader
for idx, (x, y) in enumerate(train_loader):
    print(f'Batch {idx + 1}:', x, y)

# A.7 - A typical training loop

In [ ]:
# Neural network training in PyTorch
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Logging
        print(f"Epoch: {epoch+1:d}/{num_epochs:d} | Batch {batch_idx:d}/{len(train_loader):d} | Train Loss: {loss:.5f}")
    model.eval()

## Exercise A.3 - Page 275
- Input layer: 2 x 30 + 30 = 90
- Second layer: 30 x 20 + 20 = 620
- Output layer: 20 x 2 + 2 = 42
- Total: 90 + 620 + 42 = 752 parameters.

In [ ]:
# Make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

In [ ]:
# Obtain the class membership probabilities
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

In [ ]:
# Convert class membership probabilities into class label predictions
predictions = torch.argmax(probas, dim=1)
print(predictions)

In [ ]:
# Convert class membership probabilities into class label predictions from outputs
predictions = torch.argmax(outputs, dim=1)
print(predictions)

In [ ]:
# Check correctness for each instance
predictions == y_train

In [ ]:
# See how much are correct
torch.sum(predictions == y_train)

In [ ]:
# Generalize the computation of the prediction accuracy
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    total = 0
    for idx, (features, labels) in enumerate(dataloader):
        with torch.no_grad():
            logits = model(features)
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total += len(compare)
    return (correct / total).item()

In [ ]:
# Print training accuracy
print(compute_accuracy(model, train_loader))

In [ ]:
# Print test accuracy
print(compute_accuracy(model, test_loader))

# A.8 - Saving and loading models

In [ ]:
# Save models in PyTorch
torch.save(model.state_dict(), "model.pth")

In [ ]:
# Retore the model from disk
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

# A.9 - Optimizing training performance with GPUs

## A.9.1 - PyTorch computations on GPU devices

In [ ]:
# Check availability
print(torch.cuda.is_available())

In [ ]:
# PyTorch computations on CPU
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

## A.9.2 - Single-GPU training

In [ ]:
# Train on a single GPU
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.5)
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Logging
        print(f"Epoch: {epoch+1:d}/{num_epochs:d} | Batch {batch_idx:d}/{len(train_loader):d} | Train Loss: {loss:.5f}")
    model.eval()

## Exercise A.4 - Page 282
- I don't have a GPU, so I couldn't finish this exercise.
- What I learned from Appendix C is that on V100, the computation is approximately four times faster.

In [ ]:
# Time the matrix multiplication calculation on CPU
a = torch.rand(100, 200)
b = torch.rand(200, 300)
%timeit a@b

## A.9.3 - Training with multiple GPUs
- I cannot access multiple GPUs, so I skipped this section in the Appendix A.